In [1]:
import serial
import datetime

In [2]:
# Make sure to run this script before connecting the USB

serialPort = serial.Serial(port = "COM6", baudrate = 3000000, bytesize = 8, timeout = 0, stopbits = serial.STOPBITS_ONE)
t_before = datetime.datetime.now()

In [3]:
i = 0
repeat_max = 20000
buffSizes = []
stringTotal = bytearray() #""
serialString = bytearray() #""
#buff_size = serialPort.in_waiting
while (serialPort.in_waiting >= 0) and (i < repeat_max):
    serialString = serialPort.read(serialPort.in_waiting)
    stringTotal.extend(serialString) #stringTotal += str(serialString)
    if serialPort.in_waiting > 0:
        i += 1
#    buffSizes.append(buff_size)
#    buff_size = serialPort.in_waiting
    
t_after = datetime.datetime.now()
t_diff = (t_after - t_before).total_seconds()

In [4]:
#stringTotal = stringTotal.replace("\'", "")
#stringTotal = stringTotal.replace("b", "")
print("time elapsed:", t_diff, ", bytes: ", len(stringTotal), ", packets/sec:", len(stringTotal)/t_diff, ", Incoming: ...", stringTotal[-21:])

time elapsed: 22.213024 , bytes:  5646632 , packets/sec: 254203.65997893846 , Incoming: ... bytearray(b'D\x00\x15\x8aE\x00\x15\x8aF\x00\x15\x8aG\x00\x15\x8aH\x00\x15\x8aI')


In [5]:
incomeArray = [(stringTotal[x]<<24) + (stringTotal[x+1]<<16) + (stringTotal[x+2]<<8) + (stringTotal[x+3]) for x in range (0, len(stringTotal)-4, 4)]

In [6]:
numCount = {}
for i in incomeArray:
# for i in stringTotal:
    if i in numCount.keys():
        numCount[i] += 1
    else:
        numCount[i] = 1
# print(numCount)
numCount[124]

1

In [7]:
# diff = [t - s for s, t in zip(stringTotal, stringTotal[1:])]
diff = [t - s for s, t in zip(incomeArray, incomeArray[1:])]
skipped = [i for i in diff if i not in [1]]#, -0xff]]
skippedIndices = [i for i, x in enumerate(diff) if x not in [1]]#, -0xff]]

In [8]:
print(skippedIndices)
#print(diff)
#filledBuff = [i for i in buffSizes if i != 0]
#print(filledBuff)
#print(len(buffSizes), buffSizes.count(0))

[]


In [9]:
for i in skippedIndices:
    hexSkipped = stringTotal[i - 4 : i + 6].hex()
    skipRange = [hexSkipped[i:i+2] for i in range(0,len(hexSkipped), 2)]
    a = int(skipRange[5], 16)
    b = int(skipRange[4], 16)
    print(skipRange, (a - b) & 0xff)

class ReadLine:
    def __init__(self, s, l):
        self.buf = bytearray()
        self.s = s
#        self.lim = l
    
    def readline(self):
        i = self.buf.find(b"\n")
        if i >= 0:
            r = self.buf[:i+1]
            self.buf = self.buf[i+1:]
            return r
        for k in range(5000):#while True:
            i = max(1, min(2048, self.s.in_waiting))
            data = self.s.read(i)
            i = data.find(b"\n")
            if i >= 0:
                r = self.buf + data[:i+1]
                self.buf[0:] = data[i+1:]
                return r
            else:
                self.buf.extend(data)

#serialPort = serial.Serial(port = "COM6", baudrate = 3000000, bytesize = 8, timeout = 0, stopbits = serial.STOPBITS_ONE)
rl = ReadLine(serialPort, 500000)

while True:
    print(rl.readline())